<a href="https://colab.research.google.com/github/moosakazim12/SQL-Practice/blob/main/SQL_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# Question (SELF JOIN)
# How can you output a list of all members who have recommended another member? Ensure that there are no duplicates in the list, and that results are ordered by (surname, firstname).

SELECT DISTINCT
       m.firstname,
       m.surname
FROM cd.members m
INNER JOIN cd.members r
  ON m.memid = r.recommendedby
ORDER BY m.surname, m.firstname;


print("Query executed successfully.")
eeeee

In [ ]:
# Question
# How can you produce a list of the start times for bookings by members named 'David Farrell'?
# I have taken tables and question from https://pgexercises.com/questions/joins/simplejoin.html

SELECT starttime
FROM cd.members AS m
INNER JOIN cd.bookings AS b
USING(memid)
WHERE m.firstname = 'David' AND m.surname = 'Farrell'

In [ ]:
# Question
# How can you output a list of all members, including the individual who recommended them (if any)? Ensure that results are ordered by (surname, firstname).
SELECT m1.firstname AS memfname, m1.surname AS memsname,
m2.firstname AS recfname, m2.surname AS recsname
FROM cd.members AS m1
LEFT JOIN cd.members AS m2
 ON m2.memid = m1.recommendedby
ORDER BY memsname,memfname

In [ ]:
# How can you produce a list of all members who have used a tennis court? Include in your output the name of the court, and the name of the member formatted as a single column. Ensure no duplicate data, and order by the member name followed by the facility name.

SELECT DISTINCT CONCAT(m.firstname, ' ', m.surname) AS member ,f.name AS facility
FROM
cd.members AS m
INNER JOIN cd.bookings AS b
USING(memid)
INNER JOIN cd.facilities AS f
ON b.facid = f.facid 
WHERE f.name LIKE 'Tennis%'
ORDER BY member,facility

In [ ]:
# Question
# How can you produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30? Remember that guests have different costs to members (the listed costs are per half-hour 'slot'), and the guest user is always ID 0. Include in your output the name of the facility, the name of the member formatted as a single column, and the cost. Order by descending cost, and do not use any subqueries.
SELECT
    f.name AS facility,
    CASE
        WHEN m.memid = 0 THEN 'Guest'
        ELSE m.firstname || ' ' || m.surname
    END AS member,
    CASE
        WHEN m.memid = 0 THEN b.slots * f.guestcost
        ELSE b.slots * f.membercost
    END AS cost
FROM cd.bookings b
JOIN cd.facilities f
    ON b.facid = f.facid
JOIN cd.members m
    ON b.memid = m.memid
WHERE b.starttime::date = '2012-09-14'
  AND (
        CASE
            WHEN m.memid = 0 THEN b.slots * f.guestcost
            ELSE b.slots * f.membercost
        END
      ) > 30
ORDER BY cost DESC;


In [ ]:
# Question
# How can you output a list of all members, including the individual who recommended them (if any), without using any joins? Ensure that there are no duplicates in the list, and that each firstname + surname pairing is formatted as a column and ordered.

SELECT DISTINCT m.firstname || ' ' ||  m.surname as member,
(
SELECT r.firstname || ' ' ||  r.surname as recommender
  FROM cd.members AS r
  WHERE r.memid = m.recommendedby

)

FROM cd.members AS m
ORDER BY member

In [ ]:
# Question
# The Produce a list of costly bookings exercise contained some messy logic: we had to calculate the booking cost in both the WHERE clause and the CASE statement. Try to simplify this calculation using subqueries. For reference, the question was:
# How can you produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30? Remember that guests have different costs to members (the listed costs are per half-hour 'slot'), and the guest user is always ID 0. Include in your output the name of the facility, the name of the member formatted as a single column, and the cost. Order by descending cost.

SELECT member, facility, cost FROM (
	SELECT 
		mems.firstname || ' ' || mems.surname as member,
		facs.name as facility,
		case
			when mems.memid = 0 then
				bks.slots*facs.guestcost
			else
				bks.slots*facs.membercost
		end as cost
		from
			cd.members mems
			inner join cd.bookings bks
				on mems.memid = bks.memid
			inner join cd.facilities facs
				on bks.facid = facs.facid
		where
			bks.starttime >= '2012-09-14' and
			bks.starttime < '2012-09-15'
	) as bookings
	
WHERE cost > 30
ORDER BY cost DESC;  
